# musinsa item_id 크롤링

### category code 정리  
원피스   
- 미니 원피스 : 020006

### column
- category
- sub_category
- item_id
- img_url

In [180]:
import requests
from bs4 import BeautifulSoup
from datetime import date, timedelta
import pandas as pd
import re

### 1)하위 카테고리 클릭시 나오는 프론트에있는 총 아이템 수, 총 페이지 수 먼저 크롤링

In [109]:
def category_info(sub_category):
    page_num = '1' #첫 페이지
    sort = 'pop&sub_sort'

    url = 'https://search.musinsa.com/category/'+ sub_category + \
    '?device=&d_cat_cd=020006&brand=&rate=&page_kind=search&list_kind=small&sort=' + \
    sort+'=&page='+page_num+'&display_cnt=90&sale_goods=&ex_soldout=&color=&price1=&price2=&exclusive_yn=&size=&tags=&sale_campaign_yn=&timesale_yn=&q='

    req = requests.get(url)
    raw = req.text

    html = BeautifulSoup(raw, 'html.parser')

    #해당 카테고리 옷의 전체 페이지 수 긁어오기
    total_item = html.select_one('#goods_list > div.boxed-list-wrapper > div.thumbType_box.box > span.counter.box_num_goods > label').get_text()
    total_page_n= html.select_one('span.totalPagingNum').get_text()

    total_item = re.sub("[^0-9]",'',total_item) #숫자만 남기고 제거
    print(sub_category, "카테고리","\n총 상품 수: ",total_item,"\n총 페이지 수: ",total_page_n)
    return total_item, total_page_n

In [110]:
sub_category = '020006' #하위카테고리 미니 원피스
category_info(sub_category)

020006 카테고리 
총 상품 수:  2144 
총 페이지 수:  24


('2144', '24')

### 2) page 수 만큼 반복하면서 하위 카테고리에 있는 모든 item의 정보 긁어온다.
- item_id
- img_url
- likes

In [159]:
#각 정보 담을 list 생성
item_id_c = []
img_url_c = []
page_num_c = []
like_c = []


for i in range(total_page_n):
    page_num = str(i)
    url = 'https://search.musinsa.com/category/'+category+ \
    '?device=&d_cat_cd='+'020006'+'&brand=&rate=&page_kind=search&list_kind=small&sort=pop&sub_sort=& \
    page='+page_num+'&display_cnt=90&sale_goods=&ex_soldout=&color=&price1=&price2=&exclusive_yn=&size=&tags=&sale_campaign_yn=&timesale_yn=&q='

    req = requests.get(url)
    raw = req.text

    html = BeautifulSoup(raw, 'html.parser')
    items = html.select('ul#searchList > li ')

    for item in items:
        item_id_c.append(item['data-no']) #item별 아이디를 가져옴
        img_url_c.append(item.img['data-original']) #item별 img url을 가져옴.
        #좋아요 수 숫자만 가져옴. 없으면 ''
        like_c.append(item.select_one("p.txt_cnt_like").get_text())
        page_num_c.append(i) #몇 페이지에서 가져온건지 기록

In [161]:
#몇 개 들어왔나 확인.
print(len(item_id_c), len(img_url_c),len(page_num_c),len(like_c))

2160 2160 2160 2160


In [154]:
#데이터프레임으로 만들기
df = pd.DataFrame()
df['item_id'] = item_id_c
df['img_url'] = img_url_c
df['sub_category'] = '020006'
df['category'] = '020'
df['page_num']=page_num_c
#df['like'] = like_c
df['date'] = date.today().isoformat() #'yyyy-mm-dd'

In [155]:
df.category = df.category.astype('str') #앞자리가 0이면 사라져서 다시 str으로 변환
df.sub_category = df.sub_category.astype('str')

In [167]:
df.head()

,item_id,img_url,sub_category,category,page_num,date
0,1861643,https://image.msscdn.net/images/goods_img/2021...,020006,020,0,2021-03-26
1,1861302,https://image.msscdn.net/images/goods_img/2021...,020006,020,0,2021-03-26
2,1859582,https://image.msscdn.net/images/goods_img/2021...,020006,020,0,2021-03-26
3,1859580,https://image.msscdn.net/images/goods_img/2021...,020006,020,0,2021-03-26
4,1861312,https://image.msscdn.net/images/goods_img/2021...,020006,020,0,2021-03-26


In [187]:
sub_category ='020006'
d= date.today().strftime('%Y%m%d')
f_name = 'mini_onepiece({})_{}.csv'.format(sub_category, d)
print(f_name)

mini_onepiece(020006)_20210326.csv


In [188]:
df.to_csv(f_name)